<html>
<head>
	<title></title>
</head>
<body>
<h2 style="text-align: center;line-height: 120%;">ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ВЫСШЕГО ОБРАЗОВАНИЯ НАЦИОНАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ УНИВЕРСИТЕТ «МОСКОВСКИЙ ЭНЕРГЕТИЧЕСКИЙ ИНСТИТУТ» Кафедра «РЗиАЭ»</h2>
<h2 style="text-align: center;line-height: 20%;">Лабораторная работа №3</h2>

<h3 style="text-align: center;line-height: 80%;"><em>Исследование генетического алгоритма для решения задач оптимизации </em></h3>

<p style="text-align: right;line-height: 40%;">Выполнил: студент группы Э-13м-19</p>

<p style="text-align: right;line-height: 40%;">Шмыков А.Д.</p>

<p style="text-align: right;line-height: 40%;">Проверил: Грачева Н.П.</p>
<h4 style="text-align: center;line-height: 100%;"><em> Москва, 2020</em></h3>
</body>
</html>

Libraries and data import

In [62]:
import random as r
import numpy as np
import re
import time
geneset1 = "абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
target1="узор"
target12="ГНоМ"

geneset2 = " абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯabCDEFGHIJKLMNOPQRSTUVWXYZABCDEFGHIJKLMNOPQRSTUVWXYZ"
target2="Новостное агентство NASA сообщило"

geneset3 = " абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ-.,!:;?«»—"
target3="Старик поднял глаза и произнес с расстановкою: — Здесь нет дел по крепостям. — А где же? — Это в крепостной " \
       "экспедиции. — А где же крепостная экспедиция? — Это у Ивана Антоновича. "

Угадывание слова полным перебором

In [69]:
def brute_force(target, geneset):
    start = time.perf_counter()
    res=""
    for i in range(0, len(target)):
        for j in range(0, len(geneset)):
            if geneset[j]==target[i]:
                res+=geneset[j]
                print(res)
    end = time.perf_counter()
    return end - start

brute_force(target12, geneset1)

Г
ГН
ГНо
ГНоМ


0.0002134000005753478

**Мой алгоритм использует:**

Селекцию усечением

Одноточечный кроссовер для скрещивания особей, выбранных селекцией

Битовую мутацию оставшихся особей в популяции после скрещивания с шансом мутации 1/длина цели


Functions declaration

In [56]:
def start(length, pop_size, geneset, target): #Creation of a start population only once
    population = dict()
    genes = ""
    for i in range(pop_size): 
        for j in range(0, length):
            random_gene = r.choice(geneset)
            genes += random_gene #adding symbols
        population[genes] = fitness(genes, target) # calc fitness
        genes = ""
    return population

def fitness(species , target): #Fitness count
    fit = 0
    for i in range(0, len(species)):
        if target[i] == species[i]:
            fit += 1
    return fit

def cut(population): # truncation selection
    sorted_by_value = sorted(population.items(), key=lambda kv: kv[1], reverse=True) #high vals first
    population = dict(sorted_by_value)
    parent = list(population.keys())[0] #picking species with the highest fitness
    del population[parent]
    if (list(population.keys())[0] == 0):
        parent = r.choice(list(population.keys())) #picking random
        del population[parent]

    return parent, population

def crossover(parentA, parentB,target): #1-point crossover
    if r.random() < 0.7:
        random_index = r.randint(1, len(target) - 1)
        parentASlice = parentA[:random_index]
        parentBSlice = parentB[random_index:]
        return (parentASlice + parentBSlice), (parentB[:random_index] + parentA[random_index:])
    return parentA, parentB

def mutate(population, newPopulation,target,geneset):
    for pop in population:
        for i in range(len(pop)):
            if  1 / len(target) > r.random(): #Mutation chance
              
                random_gene = r.choice(geneset)
                pop = pop[:i] + random_gene + pop[i + 1:]
        newPopulation[pop] = fitness(pop, target)


def new(population,target,geneset): #new pop creation
    newPopulation = dict()
    for i in range(0, round(pop_size * 0.25)):  # half of the population to be crossed over
        parent1, population = cut(population)
        parent2, population = cut(population)

        child1, child2 = crossover(parent1, parent2,target)
        newPopulation[child1] = fitness(child1, target)
        newPopulation[child2] = fitness(child2, target)

    mutate(population, newPopulation,target,geneset) #mutate the remaining half
    
    return newPopulation

def output(population,target):
    key_list = list(population.keys())
    val_list = list(population.values())
    output = key_list[val_list.index(len(target))]
    return output

Угадывание четырехбуквенного русскоязычного слова

In [27]:
found=False
gen=0
pop_size=len(geneset1)*5

population=start(len(target1),pop_size,geneset1,target1)
start = time.perf_counter()
while(not found):
    if (target1 in population):
        end = time.perf_counter()
        alh_time = end-start
        found=True
        res=output(population,target1)
        print(res,"gen:",gen,"time:",alh_time)
    else:
        gen+=1
        population=new(population,target1,geneset1)
        best = max ( population , key = population.get ) # best among population
     

узор gen: 9 time: 0.13685220000024856


Угадывание  фразы,  содержащей  как  слова  на  русском языке, так и слова на иностранном языке

In [30]:
found=False
gen=0
pop_size=len(geneset2)*2
population=start(len(target2),pop_size,geneset2,target2)
start = time.perf_counter()
while(not found):
    if (target2 in population):
        end = time.perf_counter()
        alh_time = end-start
        found=True
        res=output(population,target2)
        print(res,"gen:",gen,"time:",alh_time)
    else:
        gen+=1
        population=new(population,target2,geneset2)
        best = max ( population , key = population.get ) # best among population

Новостное агентство NASA сообщило gen: 478 time: 4.674147300000186


Угадывание русскоязычной фразы
Старик поднял глаза и произнес с расстановкою: — Здесь нет дел по крепостям. — А где же? — Это в крепостной экспедиции. — А где же крепостная экспедиция? — Это у Ивана Антоновича.  gen: 6331 time: 85.78700589999971


In [37]:
found=False
gen=0
pop_size=len(geneset3)*2
population=start(len(target3),pop_size,geneset3,target3)
start = time.perf_counter()
while(not found):
    if (target3 in population):
        end = time.perf_counter()
        alh_time = end-start
        found=True
        res=output(population,target3)
        print(res,"gen:",gen,"time:",alh_time)
    else:
        gen+=1
        population=new(population,target3,geneset3)
        best = max ( population , key = population.get ) # best among population

Старик поднял глаза и произнес с расстановкою: — Здесь нет дел по крепостям. — А где же? — Это в крепостной экспедиции. — А где же крепостная экспедиция? — Это у Ивана Антоновича.  gen: 6331 time: 85.78700589999971


В качестве модернизации я предложил разбиение цели на две одинаковые части, каждую из которых будет "обрабатывать" своя полупяция.

In [57]:
found=False
gen=0
pop_size=len(geneset3)*2
part1 = target3[0:len(target3)//2]
part2 = target3[len(target3)//2:]
population=start(len(part1),pop_size,geneset3,part1)
population2=start(len(part2),pop_size,geneset3,part2)
start = time.perf_counter()
while(not found):
    if (part1 in population) and (part2 in population2):
        end = time.perf_counter()
        alh_time = end-start
        found=True
        res=output(population,part1)+output(population2,part2)
        print(res,"gen:",gen,"time:",alh_time)
    else:
        gen+=1
        population=new(population,target3,geneset3)
        population2=new(population2,part2,geneset3)
        best = max ( population , key = population.get ) # best among population

Старик поднял глаза и произнес с расстановкою: — Здесь нет дел по крепостям. — А где же? — Это в крепостной экспедиции. — А где же крепостная экспедиция? — Это у Ивана Антоновича.  gen: 4548 time: 71.6469717


Старик поднял глаза и произнес с расстановкою: — Здесь нет дел по крепостям. — А где же? — Это в крепостной экспедиции. — А где же крепостная экспедиция? — Это у Ивана Антоновича.  gen: 4548 time: 71.6469717

**В результате модернизации время выполнения алгоритма уменьшилось на 15 секунд.**